In [1]:
from openai import OpenAI

# Install OpenAI library
!pip install -U -q openai tenacity

In [2]:
# Import the libraries
import pandas as pd
from IPython.display import display, HTML

# Set the display width to control the output width
pd.set_option("display.width", 100)
# Read the dataset and read the Laptop Dataset
df = pd.read_csv("laptop_data.csv")
df.head(5)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [3]:
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
import openai

openai.apikey = os.getenv("OPEN_AI_API_KEY")

In [4]:
from openai import OpenAI

client = OpenAI(api_key=openai.api_key)

In [5]:
def initialize_conversation():
    """
    Returns a list [{"role": "system", "content": system_message}]
    """

    delimiter = "####"

    example_user_dict = {
        "GPU intensity": "high",
        "Display quality": "high",
        "Portability": "low",
        "Multitasking": "high",
        "Processing speed": "high",
        "Budget": "150000",
    }

    example_user_req = {
        "GPU intensity": "_",
        "Display quality": "_",
        "Portability": "_",
        "Multitasking": "_",
        "Processing speed": "_",
        "Budget": "_",
    }

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    All the values in the example dictionary are only representative values.
    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised:
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'Budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask followup questions to understand their needs.
    You are trying to fill the values of all the keys {{'GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget'}} in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their profile clearly. \n

    {delimiter}
    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_dict}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [6]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stat

In [7]:
print(debug_conversation[0]["content"])


    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    All the values in the example dictionary are only representative values.


In [8]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay


functions_list_call = [
    {
        "name": "moderation_check",
        "description": "Check if the conversation is flagged",
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "Takes the input from the user",
                }
            },
            "required": ["user_input"],
        },
    }
]


@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format=False):
    MODEL = "gpt-3.5-turbo"

    system_message_json_output = (
        """<<. Return output in JSON format to the key output.>>"""
    )

    # If the output is required to be in JSON format
    if json_format:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]["content"] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model=MODEL,
            messages=input,
            response_format={"type": "json_object"},
            seed=1234,
        )
        output = json.loads(chat_completion_json.choices[0].message.content)
        return output
    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model=MODEL,
            messages=input,
            functions=functions_list_call,
            function_call="auto",
            seed=2345,
        )

        output = chat_completion.choices[0].message
        if output.function_call:
            return output

        # print(output.content)

    return output.content

In [9]:
# Testing the OpenAI functions defined above
input_prompt = "What is the capital of France?"
messages = [{"role": "user", "content": input_prompt}]
# system_message_json_output = """<<. Return output in JSON format.>>"""
# messages[0]['content']+=system_message_json_output
messages

[{'role': 'user', 'content': 'What is the capital of France?'}]

In [10]:
## Get LLM Outputs - normal
get_chat_completions(messages)  ## Chat Completions API

'The capital of France is Paris.'

In [11]:
## Get LLM Outputs - JSON output
get_chat_completions(
    messages, json_format=True
)  ## Chat Completions API and return in JSON format

{'output': 'Paris'}

In [12]:
data = {
    "GPU intensity": "high",
    "Display quality": "medium",
    "Portability": "low",
    "Multitasking": "medium",
    "Processing speed": "high",
    "Budget": "150000 INR",
}

In [13]:
data["GPU intensity"]

'high'

In [14]:
debug_conversation[0]["content"] + "user message"

'\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stated by user.\n    All the values

In [15]:
def iterate_llm_response(funct, debug_response, num=10):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print("-" * 50)  # Print a separator line for readability
        i += 1  # Increment the counter


# Example usage: test the consistency of responses from 'intent_confirmation_layer'
# iterate_llm_response(get_chat_completions, messages)

In [16]:
debug_user_input = "Hi, I am Anvit. I need a laptop for coding."

In [17]:
debug_conversation.append({"role": "user", "content": debug_user_input})
# print(debug_conversation[0]["content"]) # System Message
print(debug_conversation[1]["content"])  # User Input

Hi, I am Anvit. I need a laptop for coding.


In [18]:
# Let's look at the debug_conversation list
display(debug_conversation)

[{'role': 'system',
  'content': '\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as st

In [19]:
# Getting the response from the Assistant by passing the conversation to the Chat Completions API
debug_response_assistant = get_chat_completions(debug_conversation)
display(debug_response_assistant)

'Great! As a coder, you likely need a laptop that can handle coding tasks efficiently. To provide you with the best recommendation, could you please help me understand your specific requirements for the laptop? Are there any particular aspects like GPU intensity, display quality, portability, multitasking, processing speed, or budget that are important to you for coding purposes? Let me know so that I can tailor my recommendations accordingly.'

In [20]:
# Let's append this to the conversation list
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
debug_conversation

[{'role': 'system',
  'content': '\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as st

In [21]:
# Define a function called moderation_check that takes user_input as a parameter.


def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [22]:
moderation_check("I want to kill them.")

'Flagged'

In [23]:
moderation_check("She has a killer smile")

'Not Flagged'

In [24]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Not Flagged


In [25]:
moderation_check(debug_response_assistant)

'Not Flagged'

In [26]:
def intent_confirmation_layer(response_assistant):

    delimiter = "####"

    allowed_values = {"low", "medium", "high"}

    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    {{
    'GPU intensity': 'values',
    'Display quality':'values',
    'Portability':'values',
    'Multitasking':'values',
    'Processing speed':'values',
    'Budget':'number'
    }}
    The values for the keys should only be from the allowed values: {allowed_values}.
    The 'Budget' key can take only a numerical value.
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    THought 3 - Think carefully before the answering.
    """

    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"""Here is the input: {response_assistant}"""},
    ]

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        response_format={"type": "json_object"},
        seed=1234,
    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

In [27]:
input: "{{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'low'}}"


input: "{{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': '', 'Budget': '90000'}}"
# output: No


input: "Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'low','Processing speed': 'high','Budget': '200000'"
# output: Yes

In [28]:
debug_response_assistant

'Great! As a coder, you likely need a laptop that can handle coding tasks efficiently. To provide you with the best recommendation, could you please help me understand your specific requirements for the laptop? Are there any particular aspects like GPU intensity, display quality, portability, multitasking, processing speed, or budget that are important to you for coding purposes? Let me know so that I can tailor my recommendations accordingly.'

In [29]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
display(debug_confirmation)

{'result': 'No',
 'reason': 'The input does not contain the specified keys with values.'}

In [30]:
# Printing the value for better clarity
print(
    "Result:",
    debug_confirmation.get("result"),
    "\t",
    "Reason:",
    debug_confirmation.get("reason"),
)

Result: No 	 Reason: The input does not contain the specified keys with values.


In [31]:
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})

In [32]:
# Example 1 - Let's check with the confirmation_layer if all the keys are present
debug_response_assistant_1 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low',
 'Budget':'50000 INR'}}
"""
# Note that you are using double curly braces

print(debug_response_assistant_1)


Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low',
 'Budget':'50000 INR'}



In [33]:
response = intent_confirmation_layer(debug_response_assistant_1)
print(response)
response.get("result")  # Extract the result key from the dictionary

{'result': 'Yes'}


'Yes'

In [34]:
# Example 2 - Let's check confirmation_layer if all the keys are present
debug_response_assistant_2 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low'}}
"""
# Note that you are using double curly braces

print(debug_response_assistant_2)


Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low'}



In [35]:
intent_confirmation_layer(debug_response_assistant_2)

{'result': 'No', 'reason': "The 'Budget' key is missing."}

In [36]:
# Example 3 - Let's check confirmation_layer if all the keys are present
debug_response_assistant_3 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low',
 'Budget':'50000'}}
"""
# Note that you are using double curly braces

print(debug_response_assistant_3)


Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low',
 'Budget':'50000'}



In [37]:
intent_confirmation_layer(debug_response_assistant_3)

{'result': 'No',
 'reason': "The value for 'Budget' should be a numerical value, but it is provided as a string."}

In [38]:
import ast


def dictionary_present(response):
    delimiter = "####"

    user_req = {
        "GPU intensity": "high",
        "Display quality": "high",
        "Portability": "medium",
        "Multitasking": "high",
        "Processing speed": "high",
        "Budget": "200000",
    }

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
            'GPU intensity': 'low/medium/high ',
            'Display quality':'low/medium/high',
            'Portability':'low/medium/high',
            'Multitasking':'low/medium/high',
            'Processing speed':'low/medium/high',
            'Budget':'numerical value'
            }}
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output 1: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input 2: {{'GPU intensity':     'low', 'Display quality':     'low', 'Portability':    'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90,000'}}
            output 2: {{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90000'}}

            input 3: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'
            output 3: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000'}}
            {delimiter}
            Make sure not to produce any extra output or keys  in the output.
            """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"""Here is the user input: {response}"""},
    ]

    confirmation = get_chat_completions(messages, json_format=True)

    return confirmation

In [39]:
debug_response_assistant_n = """
{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low',
 'Budget':'50000'}
"""

In [40]:
response_dict_n = dictionary_present(debug_response_assistant_n)
display(response_dict_n)

{'GPU intensity': 'high',
 'Display quality': 'high',
 'Portability': 'low',
 'Multitasking': 'low',
 'Processing speed': 'low',
 'Budget': '50000'}

In [41]:
type(response_dict_n)

dict

In [42]:
debug_response_assistant_n = f"""Thank you for providing your budget.
Based on your budget of 50,000 INR, I will consider this while recommending suitable laptop options for you.
Here is the final recommendation for your laptop:
- GPU intensity: high
- Display quality: high
- Portability: low
- Multitasking: high
- Processing speed: medium
- Budget: 80,000 INR

Please note that these specifications are based on your requirements for surfing and a decent display within your budget.
Let me know if there's anything else I can assist you with!"""

In [43]:
response_dict_n = dictionary_present(debug_response_assistant_n)
display(response_dict_n)

{'GPU intensity': 'high',
 'Display quality': 'high',
 'Portability': 'low',
 'Multitasking': 'high',
 'Processing speed': 'medium',
 'Budget': '80000'}

In [44]:
type(response_dict_n)

dict

In [45]:
debug_conversation = initialize_conversation()
debug_user_input = "Hi, I am Anvit. I need a laptop for coding."
debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_completions(debug_conversation)
debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
# print(debug_confirmation)
# After a series of conversation...
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '80000'}


In [46]:
def product_map_layer(laptop_description):
    delimiter = "#####"

    lap_spec = {
        "GPU intensity": "(Type of the Graphics Processor)",
        "Display quality": "(Display Type, Screen Resolution, Display Size)",
        "Portability": "(Laptop Weight)",
        "Multitasking": "(RAM Size)",
        "Processing speed": "(CPU Type, Core, Clock Speed)",
    }

    values = {"low", "medium", "high"}

    prompt = f"""
    You are a Laptop Specifications Classifier whose job is to extract the key features of laptops and classify them as per their requirements.
    To analyze each laptop, perform the following steps:
    Step 1: Extract the laptop's primary features from the description {laptop_description}
    Step 2: Store the extracted features in {lap_spec} \
    Step 3: Classify each of the items in {lap_spec} into {values} based on the following rules: \
    {delimiter}
    GPU Intensity:
    - low: <<< if GPU is entry-level such as an integrated graphics processor or entry-level dedicated graphics like Intel UHD >>> , \n
    - medium: <<< if mid-range dedicated graphics like M1, AMD Radeon, Intel Iris >>> , \n
    - high: <<< high-end dedicated graphics like Nvidia RTX >>> , \n

    Display Quality:
    - low: <<< if resolution is below Full HD (e.g., 1366x768). >>> , \n
    - medium: <<< if Full HD resolution (1920x1080) or higher. >>> , \n
    - high: <<< if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. >>> \n

    Portability:
    - high: <<< if laptop weight is less than 1.51 kg >>> , \n
    - medium: <<< if laptop weight is between 1.51 kg and 2.51 kg >>> , \n
    - low: <<< if laptop weight is greater than 2.51 kg >>> \n

    Multitasking:
    - low: <<< If RAM size is 8 GB, 12 GB >>> , \n
    - medium: <<< if RAM size is 16 GB >>> , \n
    - high: <<< if RAM size is 32 GB, 64 GB >>> \n

    Processing Speed:
    - low: <<< if entry-level processors like Intel Core i3, AMD Ryzen 3 >>> , \n
    - medium: <<< if Mid-range processors like Intel Core i5, AMD Ryzen 5 >>> , \n
    - high: <<< if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher >>> \n
    {delimiter}

    {delimiter}
    Here is input output pair for few-shot learning:
    input 1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
    output 1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

    {delimiter}
    ### Strictly don't keep any other text in the values of the JSON dictionary other than low or medium or high ###
    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {lap_spec} for the following laptop {laptop_description}."""
    # see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input},
    ]

    response = get_chat_completions(messages, json_format=True)

    #   response = openai.chat.completions.create(
    #     model="gpt-3.5-turbo-0125",
    #     messages=[{"role": "system", "content":prompt },{"role": "user","content":input}],
    #     response_format={ "type": "json_object" }
    #     # max_tokens = 2000,
    #     )

    # response = json.loads(response)
    return response

In [47]:
laptop_description_1 = f"""
The Dell Inspiron is a versatile laptop that combines powerful performance and affordability.
It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing.
With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity.
The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience.
Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage.
Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience.
With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment.
All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.
"""

In [48]:
display(product_map_layer(laptop_description_1))

{'GPU intensity': 'medium',
 'Display quality': 'medium',
 'Portability': 'medium',
 'Multitasking': 'high',
 'Processing speed': 'medium'}

In [49]:
print(type(product_map_layer(laptop_description_1)))

<class 'dict'>


In [50]:
iterate_llm_response(product_map_layer, laptop_description_1)

Iteration: 0
{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}
--------------------------------------------------
Iteration: 1
{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}
--------------------------------------------------
Iteration: 2
{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}
--------------------------------------------------
Iteration: 3
{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}
--------------------------------------------------
Iteration: 4
{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}
--------------------------------------------------


In [51]:
import pandas as pd

laptop_df = pd.read_csv("laptop_data.csv")
laptop_df["Description"][0]

'The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.'

In [52]:
product_map_layer(laptop_df["Description"][0])

{'GPU intensity': 'medium',
 'Display quality': 'medium',
 'Portability': 'medium',
 'Multitasking': 'high',
 'Processing speed': 'medium'}

In [53]:
laptop_df.head(1)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...


In [54]:
laptop_df["laptop_feature"] = laptop_df["Description"].apply(
    lambda x: product_map_layer(x)
)

In [55]:
laptop_df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,laptop_feature
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"{'GPU intensity': 'medium', 'Display quality':..."
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"{'GPU intensity': 'high', 'Display quality': '..."
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...,"{'GPU intensity': 'medium', 'Display quality':..."
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,"{'GPU intensity': 'low', 'Display quality': 'l..."
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,"{'GPU intensity': 'high', 'Display quality': '..."


In [56]:
laptop_df["laptop_feature"][0]

{'GPU intensity': 'medium',
 'Display quality': 'medium',
 'Portability': 'medium',
 'Multitasking': 'high',
 'Processing speed': 'medium'}

In [57]:
laptop_df.to_csv("updated_laptop.csv", index=False, header=True)

In [58]:
dict1 = {
    "GPU intensity": "medium",
    "Display quality": "medium",
    "Portability": "medium",
    "Multitasking": "high",
    "Processing speed": "medium",
}
mappings = {"low": 0, "medium": 1, "high": 2}
for key, value in dict1.items():
    print("Key: ", key, " Values: ", value, "Mapping: ", mappings.get(value, -1))

Key:  GPU intensity  Values:  medium Mapping:  1
Key:  Display quality  Values:  medium Mapping:  1
Key:  Portability  Values:  medium Mapping:  1
Key:  Multitasking  Values:  high Mapping:  2
Key:  Processing speed  Values:  medium Mapping:  1


In [59]:
mappings = {"low": 0, "medium": 1, "high": 2}

In [60]:
functions_list = [
    {
        "name": "compare_laptops_with_user",
        "description": "Compare laptops with user requirement",
        "parameters": {
            "type": "object",
            "properties": {
                "user_req_string": {
                    "type": "string",
                    "description": "Takes the input from the user",
                }
            },
            "required": ["user_req_string"],
        },
    }
]
system_message = (
    f"""Your sole purpose is to call the function compare_laptops_with_user"""
)


def get_chat_completions_using_function_calling(user_req_string):
    messages_payload = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": str(user_req_string)},
    ]
    chat_completions = client.chat.completions.create(
        model="gpt-4o",
        messages=messages_payload,
        seed=2345,
        functions=functions_list,
        function_call="auto",
    )
    output = chat_completions.choices[0].message
    if output.function_call:
        function_name = output.function_call.name
        function_args = output.function_call.arguments
        print(f"Function Name: {function_name}")
        # print(function_args)
        if function_name == "compare_laptops_with_user":
            output = compare_laptops_with_user(
                ast.literal_eval(ast.literal_eval(function_args)["user_req_string"])
            )

    return output


import pandas as pd  # Importing the pandas library for data manipulation
import json
import ast


def compare_laptops_with_user(user_req_string):
    print(user_req_string)
    laptop_df = pd.read_csv("updated_laptop.csv")
    # user_requirements = response_dict_n
    user_requirements = user_req_string
    # print(laptop_df.head())
    # print('-' * 50)
    # print(user_requirements)
    # print('-' * 50)
    # Extracting user requirements from the input string (assuming it's a dictionary)
    # Since the function parameter already seems to be a string, we'll use it directly instead of extracting from a dictionary
    # Extracting the budget value from user_requirements and converting it to an integer
    budget = int(user_requirements.get("Budget", 0))
    # print('budget', budget)
    # budget
    # # Creating a copy of the DataFrame and filtering laptops based on the budget
    filtered_laptops = laptop_df.copy()
    filtered_laptops["Price"] = (
        filtered_laptops["Price"].str.replace(",", "").astype(int)
    )
    filtered_laptops = filtered_laptops[filtered_laptops["Price"] <= budget].copy()
    # print(filtered_laptops)
    # # # Mapping string values 'low', 'medium', 'high' to numerical scores 0, 1, 2
    mappings = {"low": 0, "medium": 1, "high": 2}

    # # # Creating a new column 'Score' in the filtered DataFrame and initializing it to 0
    filtered_laptops["Score"] = 0

    # # # Iterating over each laptop in the filtered DataFrame to calculate scores based on user requirements
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row["laptop_feature"]
        # print('USER_PRODUCT_MATCH_STR')
        # print(user_product_match_str)
        # laptop_values = user_product_match_str
        laptop_values = ast.literal_eval(user_product_match_str)
        # print('-' * 50)
        # print(laptop_values)
        score = 0

        #     # Comparing user requirements with laptop features and updating scores
        for key, user_value in user_requirements.items():
            # if key.lower() == 'budget':
            if key == "Budget":
                continue  # Skipping budget comparison
            laptop_value = laptop_values.get(key, None)
            # print(key, laptop_value)
            laptop_mapping = mappings.get(laptop_value, -1)
            # laptop_mapping = mappings.get(laptop_value, -1)
            # user_mapping = mappings.get(user_value, -1)
            user_mapping = mappings.get(user_value, -1)
            # print('user_mapping----->', user_mapping)
            # print('laptop_mapping', laptop_mapping)
            if laptop_mapping >= user_mapping:
                score = (
                    score + 1
                )  # Incrementing score if laptop value meets or exceeds user value

        filtered_laptops.loc[index, "Score"] = score
        # print('score-->', score)
        # Updating the 'Score' column in the DataFrame

    # Sorting laptops by score in descending order and selecting the top 3 products
    top_laptops = filtered_laptops.drop("laptop_feature", axis=1)
    top_laptops = top_laptops.sort_values("Score", ascending=False).head(3)
    top_laptops_json = top_laptops.to_json(
        orient="records"
    )  # Converting the top laptops DataFrame to JSON format

    # top_laptops
    return top_laptops_json

In [61]:
display(response_dict_n, "\n", type(response_dict_n))

{'GPU intensity': 'high',
 'Display quality': 'high',
 'Portability': 'low',
 'Multitasking': 'high',
 'Processing speed': 'medium',
 'Budget': '80000'}

'\n'

dict

In [62]:
get_chat_completions_using_function_calling(dictionary_present(response_dict_n))

Function Name: compare_laptops_with_user
{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '80000'}


'[{"Brand":"Lenovo","Model Name":"ThinkPad","Core":"Ryzen 7","CPU Manufacturer":"AMD","Clock Speed":"3.0 GHz","RAM Size":"16GB","Storage Type":"SSD","Display Type":"IPS","Display Size":"14\\"","Graphics Processor":"NVIDIA GTX","Screen Resolution":"2560x1440","OS":"Linux","Laptop Weight":"1.6 kg","Special Features":"Backlit Keyboard","Warranty":"3 years","Average Battery Life":"6 hours","Price":60000,"Description":"The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing strong processing capabilities for demanding tasks. With 16GB of RAM and an SSD, it offers smooth multitasking and fast storage access. The laptop features a 14\\" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It also comes with an NVIDIA GTX graphics card for enhanced graphical performance. Weighing just 1.6 kg, it is lightweight and highly portable. The laptop features a backlit k

In [63]:
top_3_laptops = compare_laptops_with_user(response_dict_n)

display(top_3_laptops)

{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '80000'}


'[{"Brand":"Lenovo","Model Name":"ThinkPad","Core":"Ryzen 7","CPU Manufacturer":"AMD","Clock Speed":"3.0 GHz","RAM Size":"16GB","Storage Type":"SSD","Display Type":"IPS","Display Size":"14\\"","Graphics Processor":"NVIDIA GTX","Screen Resolution":"2560x1440","OS":"Linux","Laptop Weight":"1.6 kg","Special Features":"Backlit Keyboard","Warranty":"3 years","Average Battery Life":"6 hours","Price":60000,"Description":"The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing strong processing capabilities for demanding tasks. With 16GB of RAM and an SSD, it offers smooth multitasking and fast storage access. The laptop features a 14\\" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It also comes with an NVIDIA GTX graphics card for enhanced graphical performance. Weighing just 1.6 kg, it is lightweight and highly portable. The laptop features a backlit k

In [64]:
# Get output in JSON Format
top_3_laptops_json = json.loads(top_3_laptops)
# type(top_3_laptops_json)
top_3_laptops_json

[{'Brand': 'Lenovo',
  'Model Name': 'ThinkPad',
  'Core': 'Ryzen 7',
  'CPU Manufacturer': 'AMD',
  'Clock Speed': '3.0 GHz',
  'RAM Size': '16GB',
  'Storage Type': 'SSD',
  'Display Type': 'IPS',
  'Display Size': '14"',
  'Graphics Processor': 'NVIDIA GTX',
  'Screen Resolution': '2560x1440',
  'OS': 'Linux',
  'Laptop Weight': '1.6 kg',
  'Special Features': 'Backlit Keyboard',
  'Warranty': '3 years',
  'Average Battery Life': '6 hours',
  'Price': 60000,
  'Description': 'The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing strong processing capabilities for demanding tasks. With 16GB of RAM and an SSD, it offers smooth multitasking and fast storage access. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It also comes with an NVIDIA GTX graphics card for enhanced graphical performance. Weighing just 1.6 kg, it is 

In [65]:
def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]["Score"] > 2:
            data1.append(data[i])

    return data1

In [66]:
validated_data = recommendation_validation(top_3_laptops)
display(validated_data, "\n")

[{'Brand': 'Lenovo',
  'Model Name': 'ThinkPad',
  'Core': 'Ryzen 7',
  'CPU Manufacturer': 'AMD',
  'Clock Speed': '3.0 GHz',
  'RAM Size': '16GB',
  'Storage Type': 'SSD',
  'Display Type': 'IPS',
  'Display Size': '14"',
  'Graphics Processor': 'NVIDIA GTX',
  'Screen Resolution': '2560x1440',
  'OS': 'Linux',
  'Laptop Weight': '1.6 kg',
  'Special Features': 'Backlit Keyboard',
  'Warranty': '3 years',
  'Average Battery Life': '6 hours',
  'Price': 60000,
  'Description': 'The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing strong processing capabilities for demanding tasks. With 16GB of RAM and an SSD, it offers smooth multitasking and fast storage access. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It also comes with an NVIDIA GTX graphics card for enhanced graphical performance. Weighing just 1.6 kg, it is 

'\n'

In [67]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    user_message = f""" These are the user's products: {products}"""
    conversation = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]
    # conversation_final = conversation[0]['content']
    return conversation

In [68]:
debug_conversation_reco = initialize_conv_reco(top_3_laptops)
debug_conversation_reco

[{'role': 'system',
  'content': '\n    You are an intelligent laptop gadget expert and you are tasked with the objective to     solve the user queries about any product from the catalogue in the user message     You should keep the user profile in mind while answering the questions.\n    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:\n    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>\n    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>\n\n    '},
 {'role': 'user',
  'content': ' These are the user\'s products: [{"Brand":"Lenovo","Model Name":"ThinkPad","Core":"Ryzen 7","CPU Manufacturer":"AMD","Clock Speed":"3.0 GHz","RAM Size":"16GB","Storage Type":"SSD","Display Type":"IPS","Display Size":"14\\"","Graphics Processor":"NVIDIA GTX","Screen Resolution":"2560x1440","OS":"Linux","Laptop Weight":"1.6 kg","Special Features":"Backlit Keyboard","Warranty":"3 years","Average Battery Li

In [69]:
debug_recommendation = get_chat_completions(debug_conversation_reco)
print(debug_recommendation + "\n")

### Laptop Summary:
1. Acer Predator: Intel Core i7 2.8 GHz, 16GB RAM, 17.3" IPS display, NVIDIA GTX, Windows 10, 3.2 kg, Dual Cooling Fans, 1 year warranty, 5 hours battery life, Rs 80,000
2. Lenovo ThinkPad: AMD Ryzen 7 3.0 GHz, 16GB RAM, 14" IPS display, NVIDIA GTX, Linux, 1.6 kg, Backlit Keyboard, 3 years warranty, 6 hours battery life, Rs 60,000
3. MSI GL65: Intel Core i7 2.6 GHz, 16GB RAM, 15.6" IPS display, NVIDIA GTX, Windows 10, 2.3 kg, RGB Keyboard, 2 years warranty, 4 hours battery life, Rs 55,000

How can I assist you further with these laptops?



In [70]:
response_dict_n

{'GPU intensity': 'high',
 'Display quality': 'high',
 'Portability': 'low',
 'Multitasking': 'high',
 'Processing speed': 'medium',
 'Budget': '80000'}

In [71]:
debug_conversation_reco.append(
    {"role": "user", "content": "This is my user profile" + str(response_dict_n)}
)
debug_conversation_reco.append({"role": "assistant", "content": debug_recommendation})

In [72]:
debug_user_input = "Which is ideal for travel?"

In [73]:
debug_conversation_reco.append({"role": "user", "content": debug_user_input})
debug_response_asst_reco = get_chat_completions(debug_conversation_reco)
print(debug_response_asst_reco)
display("\n" + debug_response_asst_reco + "\n")

Based on your user profile with a preference for low portability, the Lenovo ThinkPad would be more suitable for travel. It is lightweight at 1.6 kg and features a compact 14" display, making it easier to carry around than the other two laptops. Additionally, it offers a longer battery life of up to 6 hours, which can be beneficial while traveling.

If you prioritize portability for travel, the Lenovo ThinkPad would be the ideal choice among the three laptops mentioned. Let me know if you need more information or assistance on this.


'\nBased on your user profile with a preference for low portability, the Lenovo ThinkPad would be more suitable for travel. It is lightweight at 1.6 kg and features a compact 14" display, making it easier to carry around than the other two laptops. Additionally, it offers a longer battery life of up to 6 hours, which can be beneficial while traveling.\n\nIf you prioritize portability for travel, the Lenovo ThinkPad would be the ideal choice among the three laptops mentioned. Let me know if you need more information or assistance on this.\n'

In [74]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    display(introduction + "\n")

    top_3_laptops = None

    user_input = ""

    while user_input != "exit":

        user_input = input("")

        # moderation = moderation_check(user_input)
        # if moderation == 'Flagged':
        #     display("Sorry, this message has been flagged. Please restart your conversation.")
        #     break

        if top_3_laptops is None:

            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_completions(conversation)
            if not isinstance(response_assistant, str):
                # print(type(response_assistant))
                print("Your conversation is flagged! Please re-start")
                break

            confirmation = intent_confirmation_layer(response_assistant)

            print("Intent Confirmation Yes/No:", confirmation.get("result"))

            if "No" in confirmation.get("result"):
                conversation.append(
                    {"role": "assistant", "content": str(response_assistant)}
                )
                print("\n" + str(response_assistant) + "\n")

            else:
                print("\n" + str(response_assistant) + "\n")
                print("\n" + "Variables extracted!" + "\n")

                response = dictionary_present(response_assistant)

                print(
                    "Thank you for providing all the information. Kindly wait, while I fetch the products: \n"
                )
                top_3_laptops = get_chat_completions_using_function_calling(response)

                # print("Top 3 laptops are: \n",top_3_laptops)
                # for laptop in top_3_laptops:
                #     print(laptop)

                validated_reco = recommendation_validation(top_3_laptops)

                conversation_reco = initialize_conv_reco(validated_reco)

                conversation_reco.append(
                    {
                        "role": "user",
                        "content": "This is my user profile" + str(response),
                    }
                )

                recommendation = get_chat_completions(conversation_reco)

                if not isinstance(recommendation, str):
                    display(
                        "Sorry, this message has been flagged. Please restart your conversation."
                    )
                    break

                conversation_reco.append(
                    {"role": "assistant", "content": str(recommendation)}
                )

                print(str(recommendation) + "\n")
        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_completions(conversation_reco)
            if not isinstance(response_asst_reco, str):
                print(
                    "Sorry, this message has been flagged. Please restart your conversation."
                )
                break

            print("\n" + response_asst_reco + "\n")
            conversation.append({"role": "assistant", "content": response_asst_reco})

In [75]:
dialogue_mgmt_system()

'Welcome! Please share your requirements so that I can help you find the best laptop.\n\n# Tools\n\n## functions\n\nnamespace functions {\n\n// Check if the conversation is flagged\ntype moderation_check = (_: {\n// Takes the input from the user\nuser_input: string,\n}) => any;\n\n} // namespace functions\n'

Intent Confirmation Yes/No: No

Great! As a video editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high-end display for better visuals and editing. May I know if you primarily work with high-resolution media files, such as 4K videos? Understanding your file sizes will help determine the storage capacity and processing power needed. Can you please provide more details on the type of video editing work you frequently do?

Intent Confirmation Yes/No: No

Thank you for providing that information. Working with 4K videos requires a laptop with a good processor and high GPU intensity. Based on this, I can infer that you would need high GPU intensity for your video editing tasks. 

To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary 

In [76]:
gamer_conversation = [
    {
        "role": "assistant",
        "content": "Hi there! I'm an intelligent laptop gadget expert and I'm here to help you find the best laptop for your needs. Please tell me a little bit about yourself and your requirements so that I can assist you better.",
    },
    {
        "role": "user",
        "content": "Yes, I prefer gaming with my graphics at high setting",
    },
    {
        "role": "assistant",
        "content": "Thank you for letting me know. Based on your preference for high graphics settings, I would recommend a laptop with a high GPU intensity. This will ensure that you can enjoy your games with excellent visual quality. Now, I have a question to better understand your needs. Are you primarily looking for a laptop that you can easily carry around and game on the go, or do you mainly use it in a stationary location?",
    },
    {
        "role": "user",
        "content": "I prefer using my laptop at a stationary location. I also prefer a high-resolution display quality with good multitasking capabilities",
    },
    {
        "role": "assistant",
        "content": "Thank you for providing that information. Since you primarily use your laptop in a stationary location, portability may not be a top priority for you. Now, I have one more question to help me understand your requirements better. Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements.",
    },
    {"role": "user", "content": "150000"},
]

gamer_conversation_initialize = initialize_conversation()
gamer_conversation_initialize.extend(gamer_conversation)
gamer_conversation_assistant_output = get_chat_completions(
    gamer_conversation_initialize
)
gamer_response_dict = dictionary_present(gamer_conversation_assistant_output)
print(gamer_response_dict, "\n")

{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': '_', 'Budget': '150000'} 



In [77]:
gamer_tagged_dict = {
    "GPU intensity": "high",
    "Display quality": "high",
    "Portability": "low",
    "Multitasking": "high",
    "Processing speed": "high",
    "Budget": "150000",
}
print(gamer_tagged_dict)

{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': '150000'}


In [78]:
academic_conversation = [
    {
        "role": "assistant",
        "content": "Hi there! I'm an intelligent laptop gadget expert and I'm here to help you find the best laptop for your needs. Please tell me a little bit about yourself and your requirements so that I can assist you better.",
    },
    {"role": "user", "content": "i am a student and i'm looking to buy some laptops"},
    {
        "role": "assistant",
        "content": "Great! As a student, you likely have specific requirements for a laptop that will support your academic needs. Could you please provide me with more information about your requirements? Are you looking for a laptop for general use, such as browsing the internet, taking notes, and writing assignments? Or do you have any specific software or tasks that you need the laptop to handle? Understanding your needs will help me recommend the best laptop for you.",
    },
    {
        "role": "user",
        "content": "I am working as a academic researcher and I do work with softwares and programming languages such as Matlab, Python and CATIA",
    },
    {
        "role": "assistant",
        "content": "Thank you for providing that information. As an academic researcher, you require a laptop that can handle software and programming tasks efficiently. May I know if you work with large datasets or perform any intensive computations? Understanding the nature of your work will help me determine the processing power and multitasking capabilities needed for your laptop.",
    },
    {
        "role": "user",
        "content": "Yes, occasionally. I'm currently also working on a project on computer vision",
    },
    {
        "role": "assistant",
        "content": "Thank you for sharing that. Working on computer vision projects can be resource-intensive, especially when dealing with large datasets and complex algorithms. It requires a laptop with high processing power and a capable graphics card. Do you also require a high-quality display for accurate visualization of images and videos? Additionally, do you prioritize portability, as you may need to work on your projects in different locations?",
    },
    {
        "role": "user",
        "content": "I need a decent display and I also carry my laptop to college and conferences",
    },
    {
        "role": "assistant",
        "content": "Understood. Based on your requirements as an academic researcher working with software and programming languages like Matlab, Python, and CATIA, as well as computer vision projects, I would recommend the following specifications for your laptop: - GPU intensity: High (for handling computer vision tasks) - Display quality: Medium (for decent display) - Portability: Medium (since you carry your laptop to college and conferences) - Multitasking: High (to handle multiple software and programming tasks) - Processing speed: High (for efficient computation) Now, I need to know your budget for the laptop. Could you please let me know your maximum budget in INR?",
    },
    {"role": "user", "content": "approximately 100000"},
]

academic_conversation_initialize = initialize_conversation()
academic_conversation_initialize.extend(academic_conversation)
academic_conversation_assistant_output = get_chat_completions(
    academic_conversation_initialize
)
academic_response_dict = dictionary_present(academic_conversation_assistant_output)
# academic_dict = extract_dictionary_from_string(academic_response_dict)
print(academic_response_dict)

{'GPU intensity': 'high', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': 100000}


In [79]:
academic_tagged_dict = {
    "GPU intensity": "high",
    "Display quality": "medium",
    "Portability": "medium",
    "Multitasking": "medium",
    "Processing speed": "high",
    "Budget": "100000",
}
print(academic_tagged_dict)

{'GPU intensity': 'high', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'high', 'Budget': '100000'}


In [80]:
business_conversation = [
    {
        "role": "assistant",
        "content": "Hi there! I'm an intelligent laptop gadget expert and I'm here to help you find the best laptop for your needs. Please tell me a little bit about yourself and your requirements so that I can assist you better.",
    },
    {
        "role": "user",
        "content": "Hi. I am a business executive. I'm in the market to buy a laptop that fits my needs",
    },
    {
        "role": "assistant",
        "content": "Great! As a business executive, having a reliable and efficient laptop is crucial for your work. May I know what kind of tasks you primarily focus on? Are you involved in tasks such as data analysis, presentations, or multitasking between various applications? Understanding your specific work requirements will help me recommend the best laptop for you. ",
    },
    {
        "role": "user",
        "content": "I'll use my laptop for day-to-day work such as attending zoom calls with clients, preparting reports and doing market research. I also do analysis on Excel",
    },
    {
        "role": "assistant",
        "content": "Thank you for providing that information. Based on your work requirements, it seems like you need a laptop that can handle multitasking, has good processing speed, and a decent display quality for presentations and data analysis. Portability may not be a top priority since you mentioned that you primarily work from a stationary location. Do you have any specific preferences or requirements for the graphics performance of the laptop?",
    },
    {
        "role": "user",
        "content": "I require a highly portable machine as I'll be commuting to work and also travelling to client locations. Graphics is not a priority for me",
    },
    {
        "role": "assistant",
        "content": "Understood. In that case, a laptop with high portability and multitasking capabilities would be ideal for you. Since graphics performance is not a priority, we can focus more on processing speed and display quality. Do you have any specific budget in mind for your laptop purchase? Knowing your budget will help me find options that fit within your price range while meeting your specified requirements.",
    },
    {"role": "user", "content": "No budget is not a constraint. choose the best"},
    {
        "role": "assistant",
        "content": "Thank you for letting me know. In that case, I will recommend a laptop with high processing speed, good display quality, and excellent portability. This will ensure that you can efficiently handle your day-to-day work tasks, attend Zoom calls, prepare reports, and perform market research without any performance issues. Here are the specifications for the laptop I recommend for you: - GPU intensity: Low - Display quality: High - Portability: High - Multitasking: High - Processing speed: High - Budget: Not specified Please note that the budget is not specified as you mentioned that it is not a constraint. However, if you have any specific budget in mind, please let me know and I can provide you with more tailored recommendations.",
    },
    {"role": "user", "content": "okay. maximum 200000"},
]

business_conversation_initialize = initialize_conversation()
business_conversation_initialize.extend(business_conversation)
business_conversation_assistant_output = get_chat_completions(
    business_conversation_initialize
)
business_response_dict = dictionary_present(business_conversation_assistant_output)
print(business_response_dict)

{'GPU intensity': 'Low', 'Display quality': 'High', 'Portability': 'High', 'Multitasking': 'High', 'Processing speed': 'High', 'Budget': '200000'}


In [81]:
business_tagged_dict = {
    "GPU intensity": "low",
    "Display quality": "high",
    "Portability": "high",
    "Multitasking": "high",
    "Processing speed": "high",
    "Budget": "200000",
}
print(business_tagged_dict)

{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': '200000'}


In [82]:
"low" == "Low"

False

In [83]:
def evaluate_model_response(tagged_dict, model_dict):
    score = 0
    mappings = {"low": 0, "medium": 1, "high": 2}

    for key in tagged_dict.keys():
        if key == "budget":
            continue
        tagged_value = tagged_dict[key]
        model_value = model_dict[key]
        tagged_mapping = mappings.get(tagged_value, -1)
        model_mapping = mappings.get(model_value, -1)

        if model_mapping >= tagged_mapping:
            score += 1

    return score

In [84]:
gamer_score = evaluate_model_response(gamer_tagged_dict, gamer_response_dict)
print(gamer_score)
academic_score = evaluate_model_response(academic_tagged_dict, academic_response_dict)
print(academic_score)
business_score = evaluate_model_response(business_tagged_dict, business_response_dict)
print(business_score)

5
6
1
